In [1]:
import findspark
findspark.init("C:\Hadoop\spark-3.0.0-bin-hadoop3.2")
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Training').getOrCreate()

In [2]:
import sys

In [3]:
from pyspark.sql.functions import when,udf,lit,split,current_timestamp

In [4]:
import re

In [5]:
import csv

In [6]:
sources = {}
rules = {}
targets = {}

load = csv.DictReader(open("dl/sources.csv"))
for x in load:
    sources[x['source_var']] = x
    
load = csv.DictReader(open("dl/rules.csv"))
for x in load:
    rules[x['rule_name']] = x
    
load = csv.DictReader(open("dl/targets.csv"))
for x in load:
    targets[x['target_var']] = x

In [7]:
def setNewValue(severity,df,oldColumn,newColumn,default):
    
    if severity.upper() == 'CRITICAL':
        interupt = df.filter(df[newColumn]==False).count()
        if interupt:
            sys.exit()
    elif severity.upper() == 'NONE':
        result = df
        return result
    
    result = df.withColumn('severity', when(df[newColumn],'Success').otherwise(severity))
    result = result.withColumn(newColumn, when(df[newColumn],df[oldColumn]).otherwise(default))
    
    return result

def checkRegex(name,df,column,exp):
    temp = df.withColumn('ptemp', df[column])
    result = temp.withColumn(name, temp.ptemp.rlike(exp)).drop('ptemp')
    
    return result

def checkLink(name,df1,column1,df2,column2):
    
    # Rename the column in the second dataframe to avoid ambiguity
    temp = df2.withColumnRenamed(column2,'ptemp').select('ptemp')
    
    # Execute the left join to check links
    case = df1.join(temp, df1[column1] == temp.ptemp, 'left')
    
    # Create the column with true or false argument according to null values in left join
    result = case.withColumn(name, when(case.ptemp.isNull(),False).otherwise(True)).drop('ptemp')
    
    return result

# Define if the value is 4 numerical digits
def isYear(arg):
    if re.match('\d\d\d\d$', arg):
        return True
    else:
        return False

# Define if the value is a number in a 1-31 range    
def isDay(arg):    
    if re.match('^\d{1,2}$', arg):
        if int(arg)>0 and int(arg)<=31:
            return True
        else:
            return False
    else:
        return False

# Define if the value is considerable as a month and return it in a numerical format
def isMonth(arg):
    months = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']
    
    #Check if the month is in 99 format and between 1-12
    if re.match('^\d{1,2}$', arg):
        if int(arg)>=1 and int(arg)<=12:
            return arg
        else:
            return arg
    
    #Check if the month correspond to one of the preset values
    elif any(re.match('^'+arg.upper(),month) for month in months):
        month = next(i+1 for i,month in enumerate(months) if re.match('^'+arg.upper(),month))
        return "{0:0=2d}".format(month)
    else:
        return False
    
def getDate(row):    
    if isYear(row[0]) and isMonth(row[1]) and isDay(row[2]):
        return row[0]+'-'+str(isMonth(row[1]))+'-'+row[2]
        
    elif isDay(row[0]) and isMonth(row[1]) and isYear(row[2]):
        return row[2]+'-'+str(isMonth(row[1]))+'-'+row[0]
        
    elif isMonth(row[0]) and isDay(row[1]) and isYear(row[2]):
        return row[2]+'-'+str(isMonth(row[0]))+'-'+row[1]
        
    else:
        return None
getDateUDF = udf(getDate)

def transformDateFormat(name,df,column):
    case = split(column,'\W').alias(name)
    
    result = df.withColumn(name, getDateUDF(case))
    
    return result

In [8]:
df_source = {}
df = {}

# Load Dataframes
for source in sources:
    path = sources[source]['path_source'].split(',')
    table_source = sources[source]['table_source']
    
    if table_source not in df_source:
        df_source[table_source] = spark.read.csv(path, header=True, inferSchema=True)
        
    df[sources[source]['source_var']] = df_source[table_source]

In [9]:
# Load rules
for rule in rules:
    
    rule_name = rules[rule]['rule_name']
    source_var = rules[rule]['source_var']
    function_name = rules[rule]['func_name']
    arg = rules[rule]['arg']
    default = rules[rule]['default_value']
    severity = rules[rule]['severity']
    target_var = rules[rule]['target_var']
    dq_dim = rules[rule]['dq_dim']
    column = sources[source_var]['column_source']
    dataFrame = df[source_var]
    
    if function_name == 'checkRegex':
        
        df[target_var] = checkRegex(rule_name, dataFrame, column, arg)
        
    elif function_name == 'checkLink':
        
        dataFrame2 = df[arg]
        column2 = sources[arg]['column_source']
        
        df[target_var] = checkLink(rule_name,dataFrame,column,dataFrame2,column2)
        
    elif function_name == 'transformDateFormat':
        
        df[target_var] = transformDateFormat(rule_name,dataFrame,column)
   
    else: 
        break
        
    if target_var not in sources:
        sources[target_var] = sources[source_var]
        sources[target_var]['column_source'] = rule_name
            
    df[target_var] = setNewValue(severity,df[target_var],column,rule_name,default)
    df[target_var] = df[target_var].withColumn('dq_dim', lit(dq_dim))
    df[target_var] = df[target_var].withColumn('rule', lit(rule_name))

In [10]:
df['carDF_yr'].collect()[0]

Row(car_id=1, car_brand='Volv 0', car_model='XC40', year='2019', rule4_2='2019', severity='Success', dq_dim='Conformity', rule='rule4_2')

In [11]:
for x in df:
    if 'severity' in df[x].columns:
        data_source = 'SALES'
        print(x)
        dataset = sources[x]['table_source']
        key_nm = sources[x]['df_id']
        field_nm = sources[x]['column_source']
        
        
        df[x] = df[x].withColumn('processing_dttm', current_timestamp())
        df[x] = df[x].withColumn('data_source', lit(data_source))
        df[x] = df[x].withColumn('dataset', lit(dataset))
        df[x] = df[x].withColumn('data_domain', lit(dataset))
        df[x] = df[x].withColumn('key_nm', lit(key_nm))
        df[x] = df[x].withColumn('key_val', df[x][key_nm])
        df[x] = df[x].withColumn('field_nm', lit(field_nm))
        df[x] = df[x].withColumn('field_value', df[x][field_nm])
        df[x] = df[x].withColumn('dq_rule_result', df[x]['severity'])

locationDF_pc
locationDF_pn
locationDF_cn
carDF_cb
carDF_yr
customer_locationDF_ci
customer_locationDF_li
customer_locationDF_new_tdt


In [16]:
df[x].select(df[x][key_nm]).show()

+-------+
|cust_id|
+-------+
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
|     10|
|     11|
|     12|
|     13|
|     14|
|     15|
|     16|
|     17|
|     18|
|     19|
|     20|
+-------+
only showing top 20 rows



In [27]:
df[x].select('rule5_1').show()

+----------+
|   rule5_1|
+----------+
|2019-01-21|
|2020-01-19|
|2019-02-27|
|2019-09-26|
|2019-05-07|
|2019-05-17|
|2020-03-26|
|2019-10-01|
|2019-10-03|
|2020-01-12|
|2019-06-19|
|2019-07-27|
|2019-05-03|
|2019-05-31|
|2020-03-22|
|2020-01-12|
|      null|
|2019-01-27|
|2019-06-25|
|2019-05-29|
+----------+
only showing top 20 rows



In [ ]:
target_frame_mapped = ApplyMapping.apply(frame = target_frame, mappings = [
    ("processing_dttm","string","processing_dttm","string"),
    ("data_source","string","data_source","string"),
    ("dataset","string","dataset","string"),
    ("data_domain","string","data_domain","string"),
    ("key_nm","string","key_nm","string"),
    (key_nm,"bigint","key_value","bigint"),
    ("field_nm","string","field_nm","string"),
    (field_nm,"string","field_val","string"),
    ("dq_dim","string","dq_dim","string"),
    ("Dq_rule_result","string","Dq_rule_result","string")
], transformation_ctx = "target_frame_mapped")    